In [1]:
import unittest
import os,sys
os.environ["MKL_NUM_THREADS"] = "4" # export MKL_NUM_THREADS=6
import numpy as np
import logging
%load_ext line_profiler
import dask.array as da
os.chdir('../')
sys.path.append('main/')
from full_gradient import *
os.getcwd()

'/home/rrobin/Documents/Stellacage_code'

In [2]:
print(os.getcwd())
full_grad=Full_gradient('config_file/config.ini')
def f(param_array):
    (m,n,Rmn,Zmn)=full_grad.S_parametrization
    R=param_array[:len(m)]
    Z=param_array[len(m):]
    param=((m,n,R,Z))
    return full_grad.cost(param)
def gradf(param_array):
    (m,n,Rmn,Zmn)=full_grad.S_parametrization
    R=param_array[:len(m)]
    Z=param_array[len(m):]
    param=((m,n,R,Z))
    return full_grad.grad_cost(param)

/home/rrobin/Documents/Stellacage_code


In [3]:
eps=1e-9
S_param_complete=full_grad.S_parametrization
ls=len(S_param_complete[0])
        
#we apply one pertubation
perturb=(2*np.random.random(2*ls)-1)
S_param=np.concatenate((S_param_complete[2],S_param_complete[3]))
new_param=S_param+eps*perturb
t0=time.time()
old_cost=f(S_param)
new_cost=f(new_param)
gradient=gradf(S_param)
print('time used : {}s'.format(time.time()-t0))

time used : 29.55600595474243s


In [4]:
print('numerical cost obtained: {:10e} and {:10e}'.format(old_cost,new_cost))
print('numerical gradient : {:10e}'.format((new_cost-old_cost)/eps))
print('gradient computed : {:10e}'.format(np.dot(gradient,perturb)))

numerical cost obtained: 1.336885e+00 and 1.336885e+00
numerical gradient : 1.401390e-01
gradient computed : 2.973920e-01


In [3]:
%lprun -u 1 -f full_grad.cost full_grad.cost(full_grad.S_parametrization)

Timer unit: 1 s

Total time: 4.15637 s
File: main/full_gradient.py
Function: cost at line 77

Line #      Hits         Time  Per Hit   % Time  Line Contents
    77                                               def cost(self,paramS):
    78         1          0.2      0.2      5.5          S=Toroidal_surface(paramS,(self.ntheta_coil,self.nzeta_coil),self.Np)
    79         1          3.2      3.2     78.1          cost_regcoil_dic=cost_surface.cost_surface(self.config,S=S,Sp=self.Sp)
    80         1          0.0      0.0      0.0          cost_regcoil=cost_regcoil_dic['cost_B']+ self.lamb*cost_regcoil_dic['cost_J']
    81         1          0.2      0.2      5.3          T=tools.get_tensor_distance(S,self.Sp,self.rot_tensor)
    82         1          0.5      0.5     11.0          cost_distance=self.distance_cost(T,S)
    83         1          0.0      0.0      0.0          return (cost_regcoil+cost_distance)

In [4]:
%lprun -u 1 -f full_grad.grad_cost full_grad.grad_cost(full_grad.S_parametrization)

Timer unit: 1 s

Total time: 20.9241 s
File: main/full_gradient.py
Function: grad_cost at line 84

Line #      Hits         Time  Per Hit   % Time  Line Contents
    84                                               def grad_cost(self,paramS):
    85         1          0.2      0.2      1.1          S=Toroidal_surface(paramS,(self.ntheta_coil,self.nzeta_coil),self.Np)
    86         1         17.3     17.3     82.7          result=self.compute_gradient_of(S=S)
    87         1          0.0      0.0      0.0          I_vector,I_matrix=result['I1']
    88         1          0.6      0.6      2.7          X,Y=self.gradient_cost_distance(result['T'],S)
    89         1          2.8      2.8     13.1          theta,dtildetheta,dtheta,dSdtheta=S.get_theta_pertubation()
    90         1          0.1      0.1      0.3          grad=np.einsum('ija,oija,ij->o',I_vector,theta,S.dS/S.npts)+np.einsum('ijab,oijab,ij->o',I_matrix,dtildetheta,S.dS/S.npts)
    91         1          0.0      0.0      0.1

In [ ]:
import scipy.optimize
optimize_shape=scipy.optimize.minimize(f, S_param, method='CG', jac=gradf,options={'maxiter':100,'return_all':True})

tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x7fcdf968f520>>, <Task finished name='Task-162364' coro=<Worker.heartbeat() done, defined at /users/home/robin/miniconda3/envs/shape_opti/lib/python3.9/site-packages/distributed/worker.py:941> exception=RuntimeError('dictionary changed size during iteration')>)
Traceback (most recent call last):
  File "/users/home/robin/miniconda3/envs/shape_opti/lib/python3.9/site-packages/tornado/ioloop.py", line 741, in _run_callback
    ret = callback()
  File "/users/home/robin/miniconda3/envs/shape_opti/lib/python3.9/site-packages/tornado/ioloop.py", line 765, in _discard_future_result
    future.result()
  File "/users/home/robin/miniconda3/envs/shape_opti/lib/python3.9/site-packages/distributed/worker.py", line 955, in heartbeat
    executing={
  File "/users/home/robin/miniconda3/envs/shape_opti/lib/python3.9/site-packages/distributed/

In [ ]:
print(optimize_shape)